In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from google.cloud import storage
# from pictionary_ai.params import *
# from pictionary_ai.utils import *

In [5]:
# Parameters
max_length = 150
from_bucket_name = 'quickdraw-simplified-processed'
to_bucket_name = 'quickdraw-simplified-modelready'
test_split = 0.3

In [6]:
def list_blobs(bucket_name: str) -> list:
    '''
    Lists all the blobs in the bucket.
    '''
    # Initialize a client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # List the blobs in the bucket
    blobs = bucket.list_blobs()

    # Collect the names of blobs into a list
    blob_names = [blob.name for blob in blobs]

    return blob_names

In [7]:
blob_list = list_blobs(to_bucket_name)
print(len(blob_list))
blob_list

1323


['X_json.json',
 'test_10pc_The Eiffel Tower.ndjson',
 'test_10pc_The Great Wall of China.ndjson',
 'test_10pc_The Mona Lisa.ndjson',
 'test_10pc_aircraft carrier.ndjson',
 'test_10pc_airplane.ndjson',
 'test_10pc_alarm clock.ndjson',
 'test_10pc_ambulance.ndjson',
 'test_10pc_angel.ndjson',
 'test_10pc_animal migration.ndjson',
 'test_10pc_ant.ndjson',
 'test_10pc_anvil.ndjson',
 'test_10pc_apple.ndjson',
 'test_10pc_arm.ndjson',
 'test_10pc_asparagus.ndjson',
 'test_10pc_axe.ndjson',
 'test_10pc_backpack.ndjson',
 'test_10pc_banana.ndjson',
 'test_10pc_bandage.ndjson',
 'test_10pc_barn.ndjson',
 'test_10pc_baseball bat.ndjson',
 'test_10pc_baseball.ndjson',
 'test_10pc_basket.ndjson',
 'test_10pc_basketball.ndjson',
 'test_10pc_bat.ndjson',
 'test_10pc_bathtub.ndjson',
 'test_10pc_beach.ndjson',
 'test_10pc_bear.ndjson',
 'test_10pc_beard.ndjson',
 'test_10pc_bed.ndjson',
 'test_10pc_bee.ndjson',
 'test_10pc_belt.ndjson',
 'test_10pc_bench.ndjson',
 'test_10pc_bicycle.ndjson',
 'test

In [8]:
test_10_pc = [blob for blob in blob_list if blob.find('test_10pc') != -1]
len(test_10_pc)

345

In [9]:
train_10_pc = [blob for blob in blob_list if blob.find('train_10pc') != -1]
len(train_10_pc)

345

In [10]:
test = [blob for blob in blob_list if (blob.find('test_') != -1 and blob.find('test_10pc') == -1)]
len(test)

316

In [11]:
test = [blob for blob in blob_list if (blob.find('test_') != -1 and blob.find('test_10pc') == -1)]
len(test)

316

In [12]:
len(blob_list)

1323

In [13]:
345+345+249+249+2

1190

In [14]:
test

['test_The Eiffel Tower.ndjson',
 'test_The Great Wall of China.ndjson',
 'test_The Mona Lisa.ndjson',
 'test_aircraft carrier.ndjson',
 'test_airplane.ndjson',
 'test_alarm clock.ndjson',
 'test_ambulance.ndjson',
 'test_angel.ndjson',
 'test_animal migration.ndjson',
 'test_ant.ndjson',
 'test_anvil.ndjson',
 'test_apple.ndjson',
 'test_arm.ndjson',
 'test_asparagus.ndjson',
 'test_axe.ndjson',
 'test_backpack.ndjson',
 'test_banana.ndjson',
 'test_bandage.ndjson',
 'test_barn.ndjson',
 'test_baseball bat.ndjson',
 'test_baseball.ndjson',
 'test_basket.ndjson',
 'test_basketball.ndjson',
 'test_bat.ndjson',
 'test_bathtub.ndjson',
 'test_beach.ndjson',
 'test_bear.ndjson',
 'test_beard.ndjson',
 'test_bed.ndjson',
 'test_bee.ndjson',
 'test_belt.ndjson',
 'test_bench.ndjson',
 'test_bicycle.ndjson',
 'test_binoculars.ndjson',
 'test_bird.ndjson',
 'test_birthday cake.ndjson',
 'test_blackberry.ndjson',
 'test_blueberry.ndjson',
 'test_book.ndjson',
 'test_boomerang.ndjson',
 'test_bo

In [31]:
blob_list[1:5]

['test_10pc_The Eiffel Tower.ndjson',
 'test_10pc_The Great Wall of China.ndjson',
 'test_10pc_The Mona Lisa.ndjson',
 'test_10pc_aircraft carrier.ndjson']

In [42]:
import subprocess


def concatenate_files(bucket_name, source_objects, destination_object):
    # Create the gsutil command
    command = ['gsutil', 'compose'] + [f'gs://{bucket_name}/{obj}' for obj in source_objects] + [f'gs://{bucket_name}/{destination_object}']

    # Run the command
    subprocess.run(command, check=True)

# # Example usage
# if __name__ == "__main__":
#     bucket_name = 'your_bucket_name'
#     source_objects = ['file1.txt', 'file2.txt']  # List of files to concatenate
#     destination_object = 'concatenated.txt'  # Name of the concatenated file

#     concatenate_files(bucket_name, source_objects, destination_object)

In [39]:
[f'gs://{bucket_name}/{obj}' for obj in blob_list[1:5]]

['gs://your_bucket_name/test_10pc_The Eiffel Tower.ndjson',
 'gs://your_bucket_name/test_10pc_The Great Wall of China.ndjson',
 'gs://your_bucket_name/test_10pc_The Mona Lisa.ndjson',
 'gs://your_bucket_name/test_10pc_aircraft carrier.ndjson']

In [43]:
concatenate_files(to_bucket_name, blob_list[1:5], 'test_concat')

Composing gs://quickdraw-simplified-modelready/test_concat from 4 component object(s).


In [16]:
import json

# Download a blob from a bucket and store it in memory
def download_blob_to_memory(bucket_name, source_blob_name) -> list:
    '''
    Downloads a (ndjson) blob from the bucket and return json file as dict
    '''
    # Initialize a client
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # Get the blob
    blob = bucket.blob(source_blob_name)

    # Download the blob content as a string
    blob_content = blob.download_as_string()

    # Need to replace all single quotes with double quotes for processed
    # Not need once processing code has been updated
    # blob_content = str(blob_content, encoding='utf-8').replace("'",'"')

    # Load JSON from the blob contents
    json_data = json.loads(blob_content)

    # Returns a list of dictionaries - each dictionary represents one drawing
    return json_data

In [19]:
xx = download_blob_to_memory(to_bucket_name, 'test_concat')

JSONDecodeError: Extra data: line 1 column 16958096 (char 16958095)

In [18]:
type(xx)

list

In [51]:
test_concat_blob = download_blob_to_memory(to_bucket_name, 'test_concat')

In [52]:
test_concat_blob[0:100]

b'[{"key_id": "5524890817921024", "class": "The Eiffel Tower", "length": 26, "list_deltas": [[0.0561, '

: 